# Identify Business and Personal Prospects in the landing.account table 
developername ilike %Prospect% and customer_key (sf_cust_key) is null 

#### Calling configuration Notebook

In [0]:
%run "../General/NB_Configuration"

In [0]:
spark.sql(f"""use catalog {catalog}""")

In [0]:

df = spark.sql(f'''
SELECT regexp_replace(ACCOUNT.Tax_ID__c , '[^0-9]', '') AS Tax_ID__c,
       Customer_Key__c,
       ACCOUNT.Name,
       ACCOUNT.FirstName,
       ACCOUNT.LastName,
       landing.DEVELOPERNAME,
       ACCOUNT.CreatedDate
FROM {catalog}.LANDING.ACCOUNT ACCOUNT
INNER JOIN {catalog}.LANDING.RECORDTYPE landing ON ACCOUNT.RECORDTYPEID = landing.ID
WHERE landing.DEVELOPERNAME ILIKE '%Prospect%'
  AND ACCOUNT.Tax_ID__c IS NOT NULL
  AND TRIM(ACCOUNT.Tax_ID__c) != ''
  AND regexp_replace(ACCOUNT.Tax_ID__c, '[^0-9]', '')
      NOT IN ('123456789', '111111111', '999999999', '111223333', '678678678', '123456787', '999887777', '12345769')
  AND ACCOUNT.ISDELETED = FALSE
  AND (Customer_Key__c IS NULL OR TRIM(Customer_Key__c) = '')
  order by CreatedDate desc
''')

# Create a temporary view to hold the results
df.createOrReplaceTempView('vw_Prospects_LandingAccount_a')

# Display the DataFrame
display(df)


In [0]:
df = spark.sql(f'''
    SELECT TAX_ID__C, *
    FROM {catalog}.LANDING.ACCOUNT ACCOUNT
    INNER JOIN {catalog}.LANDING.RECORDTYPE RECORDTYPE ON ACCOUNT.RECORDTYPEID = RECORDTYPE.ID
    WHERE RECORDTYPE.DEVELOPERNAME IN ('Business_Customer', 'Person_Customer')
      AND ACCOUNT.ISDELETED = FALSE
    --   AND TAX_ID__C = '599061125'
''')
df.createOrReplaceTempView('vw_Prospects_LandingAccount_b')
display(df)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW vw_Prospects_LandingAccount as Select * from vw_Prospects_LandingAccount_a where 
not exists
(select * from vw_Prospects_LandingAccount_b where vw_Prospects_LandingAccount_a.TAX_ID__C = vw_Prospects_LandingAccount_b.TAX_ID__C)

# Person Prospect Journey

In [0]:
df_PersonProspectsMatched = spark.sql('''
select  customer.SF_Cust_Key as abcdkey, vw.Tax_ID__c
from vw_Prospects_LandingAccount vw
inner join (
    select
        regexp_replace(ac.DecryptedTaxIDNumber, '[^0-9]', '')  as Tax_ID
        , SF_Cust_Key
    from silver.customer_master ac
    where ac.CurrentRecord = 'Yes'

) customer ON regexp_replace(vw.Tax_ID__c , '[^0-9]', '') = regexp_replace(customer.Tax_ID , '[^0-9]', '')
''')
df_PersonProspectsMatched.createOrReplaceTempView('vw_MatchedPerson')
df_PersonProspectsMatched.display()

if that tax id only exists in landing.account on that prospect record then stamp the sf_cust_key on it

# Merge INTO LANDING 

In [0]:
%sql
MERGE INTO LANDING.ACCOUNT AS target
USING (
    SELECT Tax_ID__c, abcdkey FROM vw_MatchedPerson
) AS source
ON source.Tax_ID__c = target.Tax_ID__c
WHEN MATCHED THEN 
    UPDATE SET target.Customer_Key__c = source.abcdkey

# Check Merge Worked


In [0]:
%sql 
select Customer_Key__c, Tax_ID__c, * 
from LANDING.ACCOUNT
INNER JOIN LANDING.RECORDTYPE RECORDTYPE
ON ACCOUNT.RECORDTYPEID = RECORDTYPE.ID
WHERE RECORDTYPE.DEVELOPERNAME ilike ('%Prospect%')
AND ACCOUNT.Tax_ID__c IS NOT NULL
AND ISDELETED = FALSE
AND Customer_Key__c is not Null